<a href="https://colab.research.google.com/github/nbpyth97/Exercise/blob/master/tp1/ps2022_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Streams 2022
## TP1 - Air Quality Monitoring (airborne particulate matter)
-- version April 6 
 - updated to full dataset.
-- version April 8 
 - added code for spark streaming (unstructured)




The goal of this project is to analyze data provided by a set of air quality sensors [sds011](https://aqicn.org/sensor/sds011/pt/). The sensors present in the dataset are located in Portugal, namely in the Lisbon metro area. Each sensor provides two values: measuring particles less than 10 µm (P1) and less than 2.5 µm (P2) in μg/m³.

The sensor data, spans the first half of 2020, and is streamed of Kafka. 

Each data sample has the following schema:

sensor_id | sensor_type | location | latitude | longitude | timestamp | P1 | P2
----------|-------------|----------|----------|-----------|-----------|----|---
string  | string | string | float | float| timestamp | float | float



## Questions


1. Find the time of day with the poorest air quality, for each location. Updated daily;
2. Find the average air quality, for each location. Updated hourly;
3. Can you show any daily and/or weekly patterns to air quality?;
4. The data covers a period of extensive population confinement due to Covid 19. Can you find a signal in the data showing air quality improvement coinciding with the confinement period?

## Requeriments

1. Solve each question using one of the systems studied in the course.
2. For questions not fully specified, provide your own interpretation, given your own analysis of the data.

## Grading Criteria 

1. Bonus marks will be given for solving questions using more than one system (eg. Spark Unstructured + Spark Structured);
2. Bonus marks will be given if some kind of graphical output is provided to present the results;
3. Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline

24th April + 1 day - ***no penalty***

For each day late, ***0.5 / day penalty***. Penalty accumulates until
the grade of the assignment reaches 8.0.

---
### Colab Setup


In [8]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#@title Install PySpark
!pip install pyspark findspark --quiet
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [3]:
#@title Install & Launch Kafka
%%bash
KAFKA_VERSION=3.1.0
KAFKA=kafka_2.13-$KAFKA_VERSION
wget -q -O /tmp/$KAFKA.tgz https://dlcdn.apache.org/kafka/$KAFKA_VERSION/$KAFKA.tgz
tar xfz /tmp/$KAFKA.tgz
wget -q -O $KAFKA/config/server1.properties - https://github.com/smduarte/ps2022/raw/main/colab/server1.properties

UUID=`$KAFKA/bin/kafka-storage.sh random-uuid`
$KAFKA/bin/kafka-storage.sh format -t $UUID -c $KAFKA/config/server1.properties
$KAFKA/bin/kafka-server-start.sh -daemon $KAFKA/config/server1.properties


Formatting /tmp/kraft-combined-logs


### Air quality sensor data publisher
This a small python Kafka client that publishes a continous stream of text lines, obtained from the periodic output of the sensors.

* The Kafka server is accessible @localhost:9092 
* The events are published to the `air_quality` topic
* Events are published 3600x faster than realtime relative to the timestamp


In [4]:
#@title Start Kafka Publisher
%%bash
pip install kafka-python dataclasses --quiet
wget -q -O - https://github.com/smduarte/ps2022/raw/main/colab/kafka-tp1-logsender.tgz | tar xfz - 2> /dev/null

cd kafka-tp1-logsender
nohup python publisher.py --filename 2020-01-06_sds011-pt.csv --topic air_quality  --speedup 3600 2> publisher-error.log > publisher-out.log &

The python code below shows the basics needed to process JSON data from Kafka source using PySpark.

Spark Streaming python documentation is found [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.streaming.html)

---
#### PySpark Kafka Stream Example


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def dumpBatchDF(df, epoch_id):
    df.show(20, False)

spark = SparkSession \
    .builder \
    .appName('Kafka Spark Structured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

lines = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option('subscribe', 'air_quality') \
  .option('startingOffsets', 'earliest') \
  .load() \
  .selectExpr('CAST(value AS STRING)')


schema = StructType([StructField('timestamp', TimestampType(), True),
                     StructField('sensor_id', StringType(), True),
                     StructField('sensor_type', StringType(), True),
                     StructField('location', StringType(), True),
                     StructField('latitude', FloatType(), True),
                     StructField('longitude', FloatType(), True),
                     StructField('p1', FloatType(), True),
                     StructField('p2', FloatType(), True)])

lines = lines.select( from_json(col('value'), schema).alias('data')).select('data.*')

query = lines \
    .writeStream \
    .outputMode('append') \
    .foreachBatch(dumpBatchDF) \
    .start()

query.awaitTermination(600)
query.stop()
spark.stop()

### Spark Streaming (UnStructured) 

Latest Spark does not support Kafka sources with UnStructured Streaming.

The next cell publishes the dataset using a TCP server, running at port 7777. For this mode, there is no need to install or run Kafka, using the cell above.

The events are played faster than "realtime", at a 3600x speedup, such that 1 hour in terms of dataset timestamps is
sent in 1 second realtime, provided the machine is fast enough. As such, Spark Streaming window functions need to be sized accordingly, since a minibatch of 1 second will be
worth 1 hour of dataset events.

In [5]:
#@title Start Socket-based Publisher
%%bash

git clone https://github.com/smduarte/ps2022.git 2> /dev/null > /dev/null || git -C ps2022 pull
cd ps2022/colab/socket-tp1-logsender/

nohup python publisher.py --filename 2020-01-06_sds011-pt.csv --port 7777  --speedup 3600 2> /tmp/publisher-error.log > /tmp/publisher-out.log &

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession \
    .builder \
    .appName('Kafka Spark UnStructured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

try:
  ssc = StreamingContext(spark.sparkContext, 1)
  lines = ssc.socketTextStream('localhost', 7777) 

  lines.pprint()
    
  ssc.start()
  ssc.awaitTermination(20)  
except Exception as err:
  print(err)
ssc.stop()
spark.stop()

-------------------------------------------
Time: 2022-04-11 16:15:33
-------------------------------------------

-------------------------------------------
Time: 2022-04-11 16:15:34
-------------------------------------------

-------------------------------------------
Time: 2022-04-11 16:15:35
-------------------------------------------
25874 SDS011 13691 37.09194288 -8.68310094 19.83 14.53
20000 SDS011 10161 41.188 -8.642 109.27 69.5
27393 SDS011 14858 41.2 -8.326 348.83 198.47
18144 SDS011 14857 41.2 -8.326 408.73 156.47
25874 SDS011 13691 37.09194288 -8.68310094 19.07 13.43
20000 SDS011 10161 41.188 -8.642 111.63 68.5
18144 SDS011 14857 41.2 -8.326 204.17 93.7
27393 SDS011 14858 41.2 -8.326 314.53 177.1
25874 SDS011 13691 37.09194288 -8.68310094 15.73 13.2
20000 SDS011 10161 41.188 -8.642 98.8 66.03
...

-------------------------------------------
Time: 2022-04-11 16:15:36
-------------------------------------------
18144 SDS011 14857 41.2 -8.326 188.43 90.93
27393 SDS011 14858

Find the time of day with the poorest air quality, for each location. Updated daily

In [34]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from datetime import datetime

pollution = None

spark = SparkSession \
    .builder \
    .appName('Kafka Spark UnStructured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

try:
  ssc = StreamingContext(spark.sparkContext, 1)
  lines = ssc.socketTextStream('localhost', 7777) 

  results = lines.filter(lambda line :len(line)>0)\
  .map(lambda line:line.split(' '))\
  .map(lambda values:((values[2],datetime.fromtimestamp(float(values[-3])),float(values[-2]),float(values[-1]))))\
  #.filter(lambda x: x[-1] if x[-1]>pollution else pollution:=x[-1])

  results.pprint()
    
  ssc.start()
  ssc.awaitTermination(5)  
except Exception as err:
  print(err)
ssc.stop()
spark.stop()

-------------------------------------------
Time: 2022-04-11 18:16:24
-------------------------------------------
('13691', datetime.datetime(1969, 12, 31, 23, 59, 51, 316899), 19.83, 14.53)
('10161', datetime.datetime(1969, 12, 31, 23, 59, 51, 358000), 109.27, 69.5)
('14858', datetime.datetime(1969, 12, 31, 23, 59, 51, 674000), 348.83, 198.47)
('14857', datetime.datetime(1969, 12, 31, 23, 59, 51, 674000), 408.73, 156.47)
('13691', datetime.datetime(1969, 12, 31, 23, 59, 51, 316899), 19.07, 13.43)
('10161', datetime.datetime(1969, 12, 31, 23, 59, 51, 358000), 111.63, 68.5)
('14857', datetime.datetime(1969, 12, 31, 23, 59, 51, 674000), 204.17, 93.7)
('14858', datetime.datetime(1969, 12, 31, 23, 59, 51, 674000), 314.53, 177.1)
('13691', datetime.datetime(1969, 12, 31, 23, 59, 51, 316899), 15.73, 13.2)
('10161', datetime.datetime(1969, 12, 31, 23, 59, 51, 358000), 98.8, 66.03)
...

-------------------------------------------
Time: 2022-04-11 18:16:25
--------------------------------------

In [37]:
from datetime import datetime
  
  
timestamp = -8.68310094
print(type(timestamp))
dt_obj = datetime.fromtimestamp(1140825600)
  
print("date_time:",dt_obj)
print("type of dt:",type(dt_obj))

<class 'float'>
date_time: 2006-02-25 00:00:00
type of dt: <class 'datetime.datetime'>
